http://neuralensemble.org/docs/PyNN/examples/VAbenchmarks.html


# open h5

In [ ]:
# laod h5
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import h5py

pathName = "data.h5"

h5 = h5py.File(pathName,'r')

list(h5.keys()) #acts as python dictionary


# open pickle file

In [ ]:
import pickle 

def getPklData(pathData):
    fileHandle = open(pathData, 'rb')
    neoObj = pickle.load(fileHandle)
    segments = neoObj.segments[0]
    spikeTrains = segments.spiktrains
    return spikeTrains

pathData = "data.pkl"
spk = getPklData(pathData)


# adapt the bottom code to plot inputs

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

def plot(signal, spikes, alpha=.2):
    fig_settings = {
        'lines.linewidth': 0.5,
        'axes.linewidth': 0.5,
        'axes.labelsize': 'small',
        'legend.fontsize': 'small',
        'font.size': 14
    }
    plt.rcParams.update(fig_settings)

    #n_panels = sum(a.shape[1] for a in signal.segments[0].analogsignals) + 1
    n_panels = len(signal.segments[0].analogsignals) +1 
    fig, axs = plt.subplots(n_panels, 1, figsize=(15, 21))

    def plot_spiketrains(ax, segment):
        for spiketrain in segment.spiketrains[:100]:
            y = np.ones_like(spiketrain) * spiketrain.annotations['source_id']
            ax.plot(spiketrain, y, '.')
            ax.set_ylabel(segment.name)
            #plt.setp(plt.gca().get_xticklabels(), visible=False)

    def plot_signal(ax, signal, index, colour='b'):
        label = "Neuron %d" % signal.annotations['source_ids'][index]
        ax.plot(signal.times, signal[:, index], colour, label=label, alpha=alpha)
        ax.set_ylabel("%s (%s)" % (signal.name, signal.units._dimensionality.string))
        #plt.setp(plt.gca().get_xticklabels(), visible=False)
        #plt.legend()

    plot_spiketrains(axs[0], spikes.segments[0])
    #plot_spiketrains(axs[1], all_spikes.segments[0])
    for i_panel, array in enumerate(signal.segments[0].analogsignals):
        for i in range(array.shape[1]):
            ax = axs[i_panel+1]
            plot_signal(ax, array, i, colour='bg'[i_panel % 2])
    ax.set_xlabel("time (%s)" % array.times.units._dimensionality.string)
    #plt.setp(plt.gca().get_xticklabels(), visible=True)
    return fig, axs

fig, axs = plot(exc_v, all_spikes)

## Modifying connections

http://neuralensemble.org/docs/PyNN/connections.html


In [ ]:
for pop in ['exc', 'ext', 'inh']:
    weights, delays = connections[pop].get(['weight', 'delay'], format='array')
    print(weights.shape, delays.shape)

In [ ]:
connections['exc']

In [ ]:
connections

In [ ]:
weights.shape

In [ ]:
weights.sum(axis=0)

In [ ]:
import numpy as np
exists = ~np.isnan(weights)

number of valid connections:

In [ ]:
exists.sum(axis=0),exists.sum(axis=0).shape

In [ ]:
weights[exists][:10], weights[exists].shape

In [ ]:
weight_cp = weights.copy()
weight_cp[np.isnan(weights)] = 0.
weight_cp.sum(axis=0)

In [ ]:
weights_norm = weights * weights[exists].mean() / weight_cp.sum(axis=0)

In [ ]:
weight_cp = weights_norm.copy()
weight_cp[np.isnan(weights_norm)] = 0.

weight_cp.sum(axis=0)

setting weights:

In [ ]:
connections[pop].get?

In [ ]:
connections[pop].get

In [ ]:
connections[pop].get

In [ ]:
c_inh = connections['inh']

In [ ]:
c_inh.get('weight', format='array').shape

In [ ]:
weight_cp.shape

In [ ]:
c_inh.set(weight=weight_cp)

## applying that strategy 

In [ ]:
node_id = sim.setup(timestep=dt, min_delay=delay, max_delay=1.0, **extra)
num_process = sim.num_processes()

print("%s Creating cell populations..." % node_id)

# create a single population of neurons, and then use population views to define
# excitatory and inhibitory sub-populations
all_cells = sim.Population(n_exc + n_inh, celltype(**cell_params), label="All Cells")
exc_cells = all_cells[:n_exc]
exc_cells.label = "Excitatory cells"
inh_cells = all_cells[n_exc:]
inh_cells.label = "Inhibitory cells"

ext_stim = sim.Population(20, sim.SpikeSourcePoisson(rate=rate, duration=stim_dur), label="expoisson")
rconn = 0.01
ext_conn = sim.FixedProbabilityConnector(rconn)
ext_syn = sim.StaticSynapse(weight=0.1)

print("%s Initialising membrane potential to random values..." % node_id)
rng = NumpyRNG(seed=rngseed, parallel_safe=parallel_safe)
uniformDistr = RandomDistribution('uniform', low=v_reset, high=v_thresh, rng=rng)
all_cells.initialize(v=uniformDistr)

print("%s Connecting populations..." % node_id)
progress_bar = ProgressBar(width=100)
connector = sim.FixedProbabilityConnector(pconn, rng=rng, callback=progress_bar)
exc_syn = sim.StaticSynapse(weight=w_exc, delay=delay)
inh_syn = sim.StaticSynapse(weight=w_inh, delay=delay)

connections = {}
connections['exc'] = sim.Projection(exc_cells, all_cells, connector, exc_syn, receptor_type='excitatory')
connections['inh'] = sim.Projection(inh_cells, all_cells, connector, inh_syn, receptor_type='inhibitory')
connections['ext'] = sim.Projection(ext_stim, all_cells, ext_conn, ext_syn, receptor_type='excitatory')

# normalizing weights
for pop in ['exc', 'inh']: # , 'ext'
    # weights, delays = connections[pop].get(['weight', 'delay'], format='array')
    weight = connections[pop].get('weight', format='array')
    print(pop, weight.shape)
    weight_cp = weight.copy()
    weight_cp[np.isnan(weight)] = 0.
    weight_norm = weight_cp * (weight[~np.isnan(weight)].mean()) / weight_cp.mean(axis=0)
    print(pop, weight_norm.shape)
    connections[pop].set(weight=weight_norm)
    
# === Setup recording ==========================================================
print("%s Setting up recording..." % node_id)
all_cells.record('spikes')
exc_cells[:n_record].record(['v', 'gsyn_exc', 'gsyn_inh'])

# === Run simulation ===========================================================
print("%d Running simulation..." % node_id)
sim.run(tstop)

E_count = exc_cells.mean_spike_count()
I_count = inh_cells.mean_spike_count()

n_connections = "%d e→e,i  %d i→e,i" % (connections['exc'].size(),
                                      connections['inh'].size())

print("\n--- Vogels-Abbott Network Simulation ---")
print("Number of Neurons      : %d" % n)
print("Number of Synapses     : %s" % n_connections)
print("Excitatory conductance : %g nS" % Gexc)
print("Inhibitory conductance : %g nS" % Ginh)
print("Excitatory rate        : %g Hz" % (E_count * 1000.0 / tstop,))
print("Inhibitory rate        : %g Hz" % (I_count * 1000.0 / tstop,))

# === Finished with simulator ==================================================

sim.end()

In [ ]:
all_spikes = all_cells.get_data()
exc_v = exc_cells.get_data()

fig, axs = plot(exc_v, all_spikes)